In [2]:
import pandas as pd



In [3]:
url="https://archive.ics.uci.edu/static/public/222/data.csv"
data=pd.read_csv(url)

In [65]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day_of_week,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,may,261,1,-1,0,NaN,no
1,44,technician,single,secondary,no,29,yes,no,NaN,5,may,151,1,-1,0,NaN,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,NaN,5,may,76,1,-1,0,NaN,no
3,47,blue-collar,married,NaN,no,1506,yes,no,NaN,5,may,92,1,-1,0,NaN,no
4,33,NaN,single,NaN,no,1,no,no,NaN,5,may,198,1,-1,0,NaN,no


In [7]:
data[['y']].value_counts()

y  
no     39922
yes     5289
dtype: int64

In [83]:
data.describe()

,age,balance,day_of_week,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [88]:
data.columns[data.isnull().any()]


Index(['job', 'education', 'contact', 'poutcome'], dtype='object')

In [90]:
from sklearn.impute import SimpleImputer

In [91]:
null_threshold = 0.5
columns_to_drop = data.columns[data.isnull().mean() > null_threshold]

# Drop columns with more than 50% null values
data = data.drop(columns=columns_to_drop)

# Identify remaining columns with null values
remaining_columns = data.columns[data.isnull().any()]

# Impute categorical columns with strategy 'most_frequent'
categorical_columns = data[remaining_columns].select_dtypes(include='object').columns
for col in categorical_columns:
    imputer = SimpleImputer(strategy='most_frequent')
    data[col] = imputer.fit_transform(data[[col]])

# Impute numerical columns with strategy 'mean'
numerical_columns = data[remaining_columns].select_dtypes(exclude='object').columns
for col in numerical_columns:
    imputer = SimpleImputer(strategy='mean')
    data[col] = imputer.fit_transform(data[[col]])

In [92]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(data.drop(['y'], axis=1), data[['y']])


data =  pd.concat([X_resampled, y_resampled], axis="columns")

In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10578 entries, 21039 to 45208
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          10578 non-null  int64 
 1   job          10499 non-null  object
 2   marital      10578 non-null  object
 3   education    10123 non-null  object
 4   default      10578 non-null  object
 5   balance      10578 non-null  int64 
 6   housing      10578 non-null  object
 7   loan         10578 non-null  object
 8   contact      8367 non-null   object
 9   day_of_week  10578 non-null  int64 
 10  month        10578 non-null  object
 11  duration     10578 non-null  int64 
 12  campaign     10578 non-null  int64 
 13  pdays        10578 non-null  int64 
 14  previous     10578 non-null  int64 
 15  poutcome     2760 non-null   object
 16  y            10578 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.7+ MB


In [80]:
data.isnull().sum(axis = 0)

data.drop(columns=['job','education','contact', 'poutcome'], inplace=True)

In [72]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [73]:
from sklearn.model_selection import train_test_split

In [93]:
data = TabularDataset(data)

    # Specify the target column
label_column = 'y'

    # Train-test split with stratification
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data[label_column], random_state=42)


    # Train a tabular predictor
predictor = TabularPredictor(label=label_column).fit(train_data=train_data, presets='best_quality')

print(predictor.leaderboard(test_data, silent=True))

No path specified. Models will be saved in: "AutogluonModels\ag-20231124_002327\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1


Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231124_002327\"
AutoGluon Version:  0.8.2
Python Version:     3.10.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Disk Space Avail:   327.33 GB / 1000.15 GB (32.7%)
Train Data Rows:    8462
Train Data Columns: 15
Label Column: y
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['yes', 'no']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = yes, class 0 = no
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (yes) vs negative (no) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or sp

                     model  score_test  score_val  pred_time_test  \
0           XGBoost_BAG_L1    0.852079   0.861735        0.322088   
1      WeightedEnsemble_L2    0.852079   0.861735        0.324101   
2   NeuralNetFastAI_BAG_L1    0.851607   0.857362        0.614225   
3  RandomForestEntr_BAG_L1    0.842155   0.841527        0.185043   
4  RandomForestGini_BAG_L1    0.841682   0.842354        0.195074   
5    NeuralNetTorch_BAG_L1    0.835539   0.845781        0.311025   
6    ExtraTreesEntr_BAG_L1    0.832231   0.833845        0.242573   
7    ExtraTreesGini_BAG_L1    0.829395   0.836800        0.267909   
8    KNeighborsUnif_BAG_L1    0.752363   0.752777        0.036480   
9    KNeighborsDist_BAG_L1    0.745274   0.752186        0.033474   

   pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  \
0       0.102026   14.608495                 0.322088                0.102026   
1       0.123030   17.398911                 0.002013                0.021004 